In [1]:
import torch
from torch import nn
# import torch.nn.functional as F
# from einops import repeat
# from einops.layers.torch import Rearrange
# from vit_pytorch.vit import Transformer

# import os
# import pandas as pd

# from sklearn.model_selection import train_test_split

# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# from torch.utils.data import Dataset, DataLoader

import numpy as np

# import transformers

from torchvision.models import resnet50

# from vit_pytorch.distill import DistillableViT, DistillWrapper



In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
import datasets_utils
train_loader, val_loader, test_loader, classes , img_size = datasets_utils.get_Chaoyang_loaders(BATCH_SIZE=64, SEED=42)
# train_loader, val_loader, test_loader, classes , img_size= datasets_utils.get_CUB_loaders(BATCH_SIZE=256, SEED=42, SPLITS=[0.50,0.25,0.25])
# train_loader, val_loader, test_loader, classes , img_size = datasets_utils.get_vegetables_dataloader(BATCH_SIZE=32, SEED=42, SPLITS=[0.50,0.25,0.25])
N_CLASSES = len(classes)

c:\Users\Jacopo\miniconda3\envs\dl_refo_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
resenetModel = resnet50(pretrained = True)
resenetModel.fc = nn.Linear(resenetModel.fc.in_features, N_CLASSES)
resenetModel = resenetModel.to(device)

c:\Users\Jacopo\miniconda3\envs\dl_refo_project\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Jacopo\miniconda3\envs\dl_refo_project\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
#train loop
def train_resnet50(model,train_loader,val_loader,epochs=10,lr=1e-2):
    # model.to(device)
    current_valid_loss , best_valid_loss = float('inf'), float('inf')

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    train_loss = []
    val_loss = []
    for epoch in range(epochs):
        model.train()
        train_loss_ = []
        val_loss_ = []
        for i,(images,classes) in enumerate(train_loader):
            images = datasets_utils.convert_images_dict_to_tensor(images, resize=224)
            
            images = images.to(device)
            classes = classes.to(device)
            optimizer.zero_grad()

            outputs  = model(images)
            loss = criterion(outputs, classes)
            loss.backward()
            optimizer.step()
            train_loss_.append(loss.item())
            # #every 100 batches, print the loss
        train_loss.append(np.mean(train_loss_))
        
        model.eval()
        with torch.no_grad():
            for i,(images,classes) in enumerate(val_loader):
                images = datasets_utils.convert_images_dict_to_tensor(images, resize=224)
                    
                images = images.to(device)
                classes = classes.to(device)
                # optimizer.zero_grad()

                outputs = model(images)
                loss = criterion(outputs, classes)
                current_valid_loss = loss.item()
                val_loss_.append(loss.item())
               
            mean_loss = np.mean(val_loss_)
            current_valid_loss = mean_loss
            if current_valid_loss < best_valid_loss:
                best_valid_loss = current_valid_loss
                print(f"\nBest validation loss: {best_valid_loss}")
                print(f"\nSaving best model for epoch: {epoch+1}\n")
                torch.save(optimizer.state_dict(), 'jacoExperiments/BEST_resnet50_finetuned_sulle_cellule.pth')
            val_loss.append(mean_loss)

        print(f'Epoch: {epoch+1}, Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}')
        torch.save(model.state_dict(),f'jacoExperiments/resnet50_finetuned_sulle_cellule.pth')
        
    return train_loss,val_loss

In [6]:
# do the finetuning
train_loss2,val_loss2 = train_resnet50(resenetModel, train_loader,val_loader,epochs=500,lr=1e-4)


Best validation loss: 0.627157555686103

Saving best model for epoch: 1

Epoch: 1, Train Loss: 0.7004589578018913, Val Loss: 0.627157555686103

Best validation loss: 0.508217023478614

Saving best model for epoch: 2

Epoch: 2, Train Loss: 0.48901021291938007, Val Loss: 0.508217023478614

Best validation loss: 0.4846656123797099

Saving best model for epoch: 3

Epoch: 3, Train Loss: 0.394599812883365, Val Loss: 0.4846656123797099

Best validation loss: 0.44166891773541767

Saving best model for epoch: 4

Epoch: 4, Train Loss: 0.3344371679085719, Val Loss: 0.44166891773541767
Epoch: 5, Train Loss: 0.3047790693331368, Val Loss: 0.4444691406355964

Best validation loss: 0.39635882609420353

Saving best model for epoch: 6

Epoch: 6, Train Loss: 0.2621708895209469, Val Loss: 0.39635882609420353
Epoch: 7, Train Loss: 0.22997126511380642, Val Loss: 0.47706950704256695
Epoch: 8, Train Loss: 0.2079773612226112, Val Loss: 0.4128217366006639

Best validation loss: 0.3611361053254869

Saving best 

In [16]:
def test_resnet50(model, test_loader):
    # define loss function
    criterion = nn.CrossEntropyLoss()

    # evaluate the model
    model.eval()
    with torch.no_grad():
        test_loss = 0
        test_acc = 0
        for images, labels in test_loader:
            images = datasets_utils.convert_images_dict_to_tensor(images, resize=224)
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
               
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            test_acc += (predicted == labels).sum().item()

        # print results
        print('Test Loss: {:.4f}, Test Acc: {:.4f}'.format(test_loss/len(test_loader), test_acc/len(test_loader.dataset)))

In [17]:
test_resnet50(resenetModel, test_loader)

Test Loss: 0.9932, Test Acc: 0.8518
